In [ ]:
import pandas as pd
import pylab as pl
import researchpy as rp
from scipy import stats
import numpy as np
import itertools as it
import matplotlib.pyplot as plt

from scipy.stats import boxcox
from numpy import exp
import scipy.stats as ss
import statsmodels.api as sa
import scikit_posthocs as sp

import statsmodels.api as sm
from statsmodels.formula.api import ols
import seaborn as sns
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
from scipy import stats #for kruskal wallis ANOVA

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA


%matplotlib inline

In [ ]:
metrics_frame = pd.read_excel('Readiness Score.xlsx', sheet_name='Data')

# transform to be exponential
metrics_frame['red_score'] = exp(metrics_frame['Total Readiness Score'])
metrics_frame['red_score'] = boxcox(metrics_frame['red_score'], 0)

# power transform
#data = boxcox(data, 0)

### Exploratory data analysis

In [ ]:
count_unit = metrics_frame.groupby(['ACTUAL UNIT']).count().reset_index()
count_unit[['ACTUAL UNIT', 'Total Readiness Score']]

In [ ]:
ax = count_unit.plot.bar(x='ACTUAL UNIT',y='Full Name', rot=0)
ax.tick_params(axis='x', labelrotation=45)
ax.set_title('Count by ACTUAL UNIT')


In [ ]:
print('Total Readiness Score Overall Stats:')
print(rp.summary_cont(metrics_frame['Total Readiness Score']))
print('Total Readiness Score Grouped by ACTUAL UNIT Stats:')
print(rp.summary_cont(metrics_frame['Total Readiness Score'].groupby(metrics_frame['ACTUAL UNIT'])))



In [ ]:
metrics_frame['Total Readiness Score'].hist()
pl.suptitle("Total Readiness Score Distribution")


In [ ]:
#metrics_frame['Total Readiness Score'].hist(by=metrics_frame['ACTUAL UNIT'], 
 #                                          figsize=(20,20))
metrics_frame['red_score'].hist(by=metrics_frame['ACTUAL UNIT'], 
                                           figsize=(20,20))
    #plt.figure(figsize=(20,20))


In [ ]:
def create_subsets(groupa, groupb):
    print('comparing %s and %s:' % (groupa, groupb))
    unit_loc_a = groupa
    unit_a_df = metrics_frame.loc[metrics_frame['ACTUAL UNIT'] == unit_loc_a]
    print('Group %s with average score %s and size %s' % (groupa, 
                                                          unit_a_df['Total Readiness Score'].mean(),
                                                          len(unit_a_df)))
    unit_loc_b = groupb
    unit_b_df = metrics_frame.loc[metrics_frame['ACTUAL UNIT'] == unit_loc_b]
    print('Group %s with average score %s and size %s' % (groupb, 
                                                          unit_b_df['Total Readiness Score'].mean(),
                                                          len(unit_b_df)))
    return unit_a_df, unit_b_df

In [ ]:

def run_t_test(unit_a, unit_b):
    t2, p2 = stats.ttest_ind(unit_a['Total Readiness Score'], unit_b['Total Readiness Score'])
    print('T value: {}'.format(t2))
    print('P value: {}'.format(p2))
    return t2,p2

In [ ]:
metrics_frame['ACTUAL UNIT'].unique()

In [ ]:
t_list = []
p_list = []
group_list = []
group1_vals = []
group2_vals = []
group1_mean = []
group2_mean = []
group1_size = []
group2_size = []
group1_std = []
group2_std = []
combinations = list(it.combinations(metrics_frame['ACTUAL UNIT'].unique(),2))
for group1, group2 in combinations: 
    group1_df, group2_df = create_subsets(group1, group2)
    t, p = run_t_test(group1_df, group2_df)
    t_list.append(t)
    p_list.append(p)
    group_list.append((group1,group2)),
    group1_vals.append(group1),
    group2_vals.append(group2),
    group1_mean.append(group1_df['Total Readiness Score'].mean()),
    group2_mean.append(group2_df['Total Readiness Score'].mean()),
    group1_size.append(len(group1_df)),
    group2_size.append(len(group2_df)),
    group1_std.append(group1_df['Total Readiness Score'].std()),
    group2_std.append(group2_df['Total Readiness Score'].std())

In [ ]:
t_test_frame = pd.DataFrame({'group':group_list,'t_value':t_list,'p_value':p_list,
                            'group1':group1_vals,'group1_size':group1_size,'group1_mean':group1_mean,
                            'group1_std':group1_std,'group2':group2_vals,'group2_size':group2_size,
                             'group2_mean':group2_mean, 'group2_std':group2_std,})

In [ ]:
boxlot = t_test_frame.boxplot(column=(['t_value']))

In [ ]:
#t_test_frame.to_csv(r'C:\Users\sarah\Documents\python\readiness_score_output\t_test_frame_CITY_SECTION.csv')

In [ ]:
t_test_frame.head()

In [ ]:
#metrics_frame['CITY/SECTION'].nunique()
#metrics_frame['trs'] = metrics_frame['Total Readiness Score']
metrics_frame['trs'] = metrics_frame['red_score']

#metrics_frame['city_section'] = metrics_frame['CITY/SECTION']
metrics_frame['actual_unit'] = metrics_frame['ACTUAL UNIT']


### One Way ANOVA 

In [ ]:
mod = ols('trs ~ actual_unit', data=metrics_frame).fit()

In [ ]:
mod.summary()

In [ ]:
aov_table = sm.stats.anova_lm(mod, type=2)

In [ ]:
print(aov_table)

In [ ]:
def anova_table(aov):
    aov['mean_sq'] = aov[:]['sum_sq']/aov[:]['df']
    
    aov['eta_sq'] = aov[:-1]['sum_sq']/sum(aov['sum_sq'])
    
    aov['omega_sq'] = (aov[:-1]['sum_sq']-(aov[:-1]['df']*aov['mean_sq'][-1]))/(sum(aov['sum_sq'])+aov['mean_sq'][-1])
    
    cols = ['sum_sq', 'df', 'mean_sq', 'F', 'PR(>F)', 'eta_sq', 'omega_sq']
    aov = aov[cols]
    return aov

anova_table(aov_table)

### Check ANOVA Assumptions

In [ ]:
#when working with linear regression and ANOVA models, the assumptions pertain to the residuals 
#and not the variables themselves. 
#Using Statsmodels, we can use the diagnostics that is already provided
mod.diagn
#Jarque-Bera (jb; jbpv is p-value) tests the assumption of normality
#Omnibus (omni; omnipv is p-value) tests the assumption of homogeneity of variance
#Condition Number (condno) assess multicollinearity. 
#Condition Number values over 20 are indicative of multicollinearity.


In [ ]:
# Homogeneity of Variance Check using Levene's
#Levene’s test for homogeneity of variance is not significant 
#which indicates that the groups have equal variances.
stats.levene(metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'BOSTON'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'CINO(IAAG)'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'CMD GRP'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'HHD'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'SUPPORT'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'MNT VIEW'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'AUSTIN'])

In [ ]:
#sns.residplot(metrics_frame.actual_unit, metrics_frame.trs.astype('float'), lowess=True, color="g")
mod.resid.hist()

In [ ]:
# Residual Normality check
#tested on the residuals as a whole which is how the diagnostic information provided by statsmodels tests
#the residuals. One could use the Jarque-Bera test provided, or one could use Shapiro or others.
stats.shapiro(mod.resid)


## ANOVA Post-Hoc Analysis

### Tukey All pairwise comparison

In [ ]:
mc = MultiComparison(metrics_frame['trs'],metrics_frame['actual_unit'])
tukey_result = mc.tukeyhsd(alpha=0.10)
print(tukey_result)
print('Unique actual_unit groups: {}'.format(mc.groupsunique))

### non parametric Kruskal-Wallis ANOVA

In [ ]:
H, p = ss.kruskal(metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'BOSTON'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'CINO(IAAG)'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'CMD GRP'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'HHD'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'SUPPORT'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'MNT VIEW'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'AUSTIN'])

In [ ]:
print("h: {}".format(H))
print("p: {}".format(p))

In [ ]:
#Due to the assumption that H has a chi square distribution, the number of samples in each group 
#must not be too small.
#A typical rule is that each sample must have at least 5 measurements.
#This test is an alternative to One way ANOVA when the data violates the 
#assumptions of normal distribution and when the sample size is too small.

stats.kruskal(metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'BOSTON'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'CINO(IAAG)'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'CMD GRP'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'HHD'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'SUPPORT'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'MNT VIEW'],
             metrics_frame['Total Readiness Score'][metrics_frame['ACTUAL UNIT'] == 'AUSTIN'])

#if there are 3 or more independent comparison groups with 5 or more observations in each group 
#then the test statistic H approximates a chi-square distribution with k-1 degree of freedom. 
#Therefore, in such a case, you can find the critical value of the test in the chi-square distribution 
#table for critical values.

### Non-Parametric Post Hoc Analysis


In [ ]:
#sp.posthoc_nemenyi(metrics_frame, val_col='Total Readiness Score', group_col='ACTUAL UNIT')
sp.posthoc_dunn(metrics_frame, val_col='Total Readiness Score', group_col='ACTUAL UNIT')

### Unspervised Clustering 

* Feature correlation
* Silhoutte score calculation
* k means clustering
* cluster analysis

In [ ]:
#metrics_frame.columns #return the columns available in the dataframe
feature_columns = ['ACTUAL UNIT','PHA Due.1', 'Dental Due.1', 'Eval Due.1', 'APFT Due.1', 'DD93 Due.1',
'SGLV Due.1', 'PRR Due.1', 'Medical Score', 'Eval Score',
'Soldier Skill Score', 'Admin Score', 'Total Readiness Score']


In [ ]:
metrics_frame_subset = metrics_frame[feature_columns]

In [ ]:
# raw metrics
corr = metrics_frame_subset.corr()
corr.style.background_gradient(cmap='coolwarm')


In [ ]:
#sns.plot.show()

In [ ]:
metrics_frame_subset_grouped = metrics_frame_subset.groupby('ACTUAL UNIT')[['PHA Due.1', 'Dental Due.1', 'Eval Due.1', 'APFT Due.1', 'DD93 Due.1',
'SGLV Due.1', 'PRR Due.1', 'Medical Score', 'Eval Score',
'Soldier Skill Score', 'Admin Score']].mean()

In [ ]:
metrics_frame_subset_grouped.plot(subplots=True,layout=(4,5), figsize=(20,20))
#plt.tight_layout()
#plt.tick_params(labelcolor='none', top='off', bottom='off', left='off', right='off')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.show()

In [ ]:

corr = metrics_frame_subset_grouped.corr()
corr.style.background_gradient(cmap='coolwarm')

In [ ]:
sns.pairplot(metrics_frame_subset_grouped)


In [ ]:
x = StandardScaler().fit_transform(metrics_frame_subset_grouped[metrics_frame_subset_grouped.columns[1:]])

In [ ]:
range_n_clusters = list (range(2,7))
print ("Number of clusters from 2 to 9: \n", range_n_clusters)
silhouette_score_values=list()

for n_clusters in range_n_clusters:
    clusterer = KMeans(n_clusters=n_clusters)
    preds = clusterer.fit_predict(x)
    centers = clusterer.cluster_centers_

    score = silhouette_score(x, preds)
    print("For n_clusters = {}, silhouette score is {})".format(n_clusters, score))
    silhouette_score_values.append(silhouette_score(x, preds,metric='euclidean', sample_size=None, random_state=None))



In [ ]:
#plt.plot(range_n_clusters, silhouette_score_values)
plt.clf()
title_obj =plt.title("Silhouette score values vs Numbers of Clusters ")
plt.setp(title_obj , color='w')         #set the color of title to white
#fig = plt.figure()
#ax = fig.add_subplot(111)
#ax = fig.add_subplot(111)
#ax.set_xlabel('n-clusters')
#ax.set_ylabel('Silhouette Score')
plt.rc_context({'ytick.color':'white','xtick.color':'white'})
#ax.xaxis.label.set_color('white')
#ax.yaxis.label.set_color('white')

#ax.tick_params(axis='x', colors='white')
#ax.tick_params(axis='y', colors='white')
plt.plot(range_n_clusters, silhouette_score_values)

plt.show()
Optimal_NumberOf_Components=range_n_clusters[silhouette_score_values.index(max(silhouette_score_values))]
print("Optimal number of components is:")
print(Optimal_NumberOf_Components)

In [ ]:
clusterer = KMeans(n_clusters=Optimal_NumberOf_Components)
preds = clusterer.fit_predict(metrics_frame_subset_grouped[metrics_frame_subset_grouped.columns[1:]])
metrics_frame_subset_grouped['cluster'] = preds

In [ ]:
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['principal component 1', 'principal component 2'])

In [ ]:
fig = plt.figure(figsize = (8,8))
ax = fig.add_subplot(1,1,1) 
ax.set_xlabel('Principal Component 1', fontsize = 15)
ax.set_ylabel('Principal Component 2', fontsize = 15)
ax.set_title('2 component PCA', fontsize = 20)
#targets = ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica']
#colors = ['r', 'g', 'b','y']
#for target, color in zip(targets,colors):
#indicesToKeep = finalDf['week'] == target
#finalDf.loc[finalDf.week, 'principal component 1']
            #   , finalDf.loc[finalDf.week, 'principal component 2']

LABEL_COLOR_MAP = {0 : 'r',
               1 : 'b',
               2:'g',
               3:'y'
               }

label_color = [LABEL_COLOR_MAP[l] for l in metrics_frame_subset_grouped.cluster]
ax.xaxis.label.set_color('white')
ax.yaxis.label.set_color('white')
ax.title.set_color('white')
ax.tick_params(axis='x', colors='white')
ax.tick_params(axis='y', colors='white')
ax.scatter(principalDf['principal component 1']
               ,principalDf['principal component 2']
               , s = 50, c=label_color)
#ax.legend(targets)
ax.grid()

In [ ]:
#fig = plt.figure(figsize = (20,20))
metrics_frame_subset_grouped.boxplot(by='cluster', figsize=(20,20))
